# Основы программирования в Python 

*Алла Тамбовцева*

## Практикум 13. Парсинг с `BeautifulSoup`: тэги и атрибуты

Мы уже немного познакомились со структурой html-страниц и поиском информации по тэгам, теперь попробуем выгрузить информацию из более интересной страницы, а точнее, с сайта [nplus1.ru](https://nplus1.ru/). Наша задача – выгрузить недавние новости в датафрейм `pandas`, чтобы потом сохранить все в файл Excel.

Для работы нам снова понадобится модуль `requests` для отправки запросов, для «подключения» к странице и получения ее содержимого в виде строки, и функция `BeautifulSoup` из библиотеки `bs4` для удобного поиска по полученной строке:

In [1]:
import requests
from bs4 import BeautifulSoup

Сохраним ссылку на главную страницу в переменную `main` и отправим запрос к ней с помощью функции `get()` из `requests`:

In [2]:
main = "https://nplus1.ru/"
page = requests.get(main)

Заберём исходный код страницы и преобразуем строку с ним в объект `BeautifulSoup`:

In [3]:
soup = BeautifulSoup(page.text)

Чтобы сгрузить все новости с главной страницы сайта, нужно собрать все ссылки на страницы с этими новостями. Ссылки в html-файле всегда заключены в тэг `<a></a>` и имеют атрибут `href`. Найдем кусочки кода HTML, соответствующие всем ссылкам на главной странице сайта:

In [4]:
links_raw = soup.find_all("a") 
links_raw[10:20]  # несколько штук для примера

[<a class="hover:underline transition-colors duration-75" href="/search/empty/768">Генетика</a>,
 <a class="hover:underline transition-colors duration-75" href="/search/empty/890">Математика</a>,
 <a class="hover:underline transition-colors duration-75" href="/search/empty/871">Космонавтика</a>,
 <a class="hover:underline transition-colors duration-75" href="/search/empty/876">Археология</a>,
 <a class="hover:underline transition-colors duration-75" href="/search/empty/775">Нейронауки</a>,
 <a class="hover:underline transition-colors duration-75" href="/search/empty/767">На мышах</a>,
 <a class="hover:underline transition-colors duration-75" href="/search/empty/771">Звук</a>,
 <a class="hover:underline transition-colors duration-75" href="/search/empty/772">Красота</a>,
 <a class="hover:underline transition-colors duration-75" href="/search/empty/778">Научные закрытия</a>,
 <a class="hover:underline transition-colors duration-75" href="/search/empty/917">ИИ спешит на помощь</a>]

Каждый элемент возвращаемого списка имеет тип `BeautifulSoup` и структуру, очень похожую на словарь. Например, ссылка `<a class="hover:underline transition-colors duration-75" href="/search/empty/768">Генетика</a>` изнутри выглядит как словарь следующего вида:

    {'href' : '/search/empty/768', 
     'class' : 'hover:underline transition-colors duration-75'}.
    
Как мы помним, значение по ключу из словаря можно вызвать с помощью метода `.get()`. Давайте извлечем значения по ключу `href` из каждого элемента списка `links`:

In [5]:
links = [li.get("href") for li in links_raw] 
links[10:20]  # несколько штук для примера

['/search/empty/768',
 '/search/empty/890',
 '/search/empty/871',
 '/search/empty/876',
 '/search/empty/775',
 '/search/empty/767',
 '/search/empty/771',
 '/search/empty/772',
 '/search/empty/778',
 '/search/empty/917']

Ссылок в списке выше много. Но нам нужны только новости – ссылки, которые начинаются с `https://nplus1.ru/news`. Создадим пустой список `news` и будем добавлять в него только ссылки, которые удовлетворяют этому условию.

In [6]:
news = []
for li in links:
    if "https://nplus1.ru/news/" in li:
        news.append(li)

In [7]:
news[0:10]

['https://nplus1.ru/news/2015/09/21/editor-thy-name',
 'https://nplus1.ru/news/2023/05/26/elephants-and-zoo-visitors',
 'https://nplus1.ru/news/2023/05/29/roman-beirut',
 'https://nplus1.ru/news/2023/05/29/patchouli',
 'https://nplus1.ru/news/2023/05/27/hakuto-r-fail-soft',
 'https://nplus1.ru/news/2023/05/27/dcm-induced-ozone-depletion',
 'https://nplus1.ru/news/2023/05/27/deomyinae',
 'https://nplus1.ru/news/2023/05/27/earlist-iron-age-house',
 'https://nplus1.ru/news/2023/05/27/higgs-z-gamma',
 'https://nplus1.ru/news/2023/05/27/leonor-of-castile']

Первая ссылка ведет не совсем на новость, скорее, на объявление, поэтому давайте ее уберем:

In [8]:
news = news[1:] 

Теперь наша задача сводится к следующему: изучить одну страницу с новостью, научиться из нее вытаскивать текст и всю необходимую информацию, а потом применить весь набор действий к каждой ссылке из `news` в цикле. Посмотрим на новость с индексом 0, у вас может быть другая, новости обновляются.

In [9]:
link0 = news[0]
print(link0)

https://nplus1.ru/news/2023/05/26/elephants-and-zoo-visitors


### Задача 1

Отправьте запрос к странице по ссылке `link0` с одной новостью, получите результат в виде объекта `BeautifulSoup` и сохраните его как `soup0`.

In [10]:
page0 = requests.get(link0)
soup0 = BeautifulSoup(page0.text)

### Задача 2

Найдите заголовок новости и сохраните его в переменную `title`.

In [11]:
title = soup0.find("title").text

### Задача 3

Найдите имя автора новости и дату её публикации. Сохраните их в `author` и `date` соответственно.

In [12]:
date = soup0.find("meta", {"itemprop" : "datePublished"}).get("content") 
author = soup0.find("meta", {"name" : "author"}).get("content") 
print(date, author)

2023-05-26 Екатерина Рощина


### Задача 4

Найдите сложность новости и рубрики, к которым она относится. Сохраните сложность в переменную `diffc`. Рубрики сначала можно сохранить в список, а затем его элементы объединить в одну строку `rubs`.

**Подсказка:** чтобы упростить себе жизнь, найдите сначала раздел (`div`), в котором на странице хранится дата и время публикации, сложность и рубрики, а затем выполняйте поиск в рамках этого раздела.

In [14]:
div = soup0.find("div", 
                 {"class" : "flex flex-wrap lg:mb-10 gap-2 text-tags xl:pr-9"})
diffc = div.find_all("span")[3].text
rubs_raw = div.find_all("span")[4:] 
rubs_L = [r.text for r in rubs_raw] 
rubs = ", ".join(rubs_L)
print(rubs)

Зоология, Психология


### Задача 5

Соберите из абзацев текст новости и сохраните его в переменную `text`. Избавьтесь от постронних символов (`\xa0`, `\n`) в тексте. 

In [15]:
pars_raw = soup0.find_all("p", {"class" : "mb-6"}) 
pars = [p.text for p in pars_raw] 
text = " ".join(pars)
text = text.replace("\xa0", " ").replace("\n", " ")
print(text)

А вот амфибии не в восторге Чаще всего животные в зоопарках относятся к посетителям нейтрально: присутствие или отсутствие гостей не меняет привычного поведения животных или их физиологических показателей. Однако некоторым везет больше — например, слоны лучше всего реагируют на людей и не испытывают их отрицательного влияния. А вот амфибии в этом вопросе — на другом полюсе. К таким выводам пришли ученые, которые проанализировали 105 научных исследований, посвященных влиянию посетителей зоопарка на их обитателей (исключая приматов). Исследование опубликовано в журнале Animals. Посетители зоопарка по-разному влияют на его обитателей. Наличие или отсутствие людей, расстояние, на которое они приближаются к животным, их поведение, создаваемый шум, форма взаимодействия с животными и даже исходящие от них запахи могут быть источником стресса для животных. В ответ на такой стресс меняются реакции животных, например, они становятся менее активны, чаще избегают встречи с посетителями, а сотрудни

### Задача 6

Напишите функцию `get_news()`, которая принимает на вход ссылку на страницу с одной новостью, а возвращает список из следующих характеристик: имя автора, дата публикации, сложность новости, рубрики, текст новости.

In [17]:
def get_news(link0):

    page0 = requests.get(link0)
    soup0 = BeautifulSoup(page0.text)
    title = soup0.find("title").text
    date = soup0.find("meta", {"itemprop" : "datePublished"}).get("content") 
    author = soup0.find("meta", {"name" : "author"}).get("content") 
    div = soup0.find("div", 
                 {"class" : "flex flex-wrap lg:mb-10 gap-2 text-tags xl:pr-9"})
    diffc = div.find_all("span")[3].text
    rubs_raw = div.find_all("span")[4:] 
    rubs_L = [r.text for r in rubs_raw] 
    rubs = ", ".join(rubs_L)
    pars_raw = soup0.find_all("p", {"class" : "mb-6"}) 
    pars = [p.text for p in pars_raw] 
    text = " ".join(pars)
    text = text.replace("\xa0", " ").replace("\n", " ")
    
    return [title, author, date, diffc, rubs, text]

Отлично! Теперь осталось применить функцию ко всем ссылкам в списке `news`. Чтобы сайт не понял, что мы его автоматически грабим, будем выгружать новости постепенно – с задержкой в 1.5 секунды. Импортируем для этого функцию `sleep` :

In [18]:
from time import sleep

Теперь будем применять функцию в цикле к каждой ссылке в `news`, только с одним дополнением – добавленной конструкцией `try-except`, которая позволит продолжать исполнение цикла, если при применении функции Python столкнулся с ошибкой любого вида:

In [19]:
info = []
for n in news:
    # пробуй исполнить следующий код
    try:
        res = get_news(n)
        info.append(res)
        print(n)
    # если он вызвал ошибку, печатай сообщение и иди дальше
    except:
        print("Something went wrong")
        print(n)
    sleep(1.2)

https://nplus1.ru/news/2023/05/26/elephants-and-zoo-visitors
https://nplus1.ru/news/2023/05/29/roman-beirut
https://nplus1.ru/news/2023/05/29/patchouli
https://nplus1.ru/news/2023/05/27/hakuto-r-fail-soft
https://nplus1.ru/news/2023/05/27/dcm-induced-ozone-depletion
https://nplus1.ru/news/2023/05/27/deomyinae
https://nplus1.ru/news/2023/05/27/earlist-iron-age-house
https://nplus1.ru/news/2023/05/27/higgs-z-gamma
https://nplus1.ru/news/2023/05/27/leonor-of-castile
https://nplus1.ru/news/2023/05/26/virgin-orbit-end
https://nplus1.ru/news/2023/05/26/hyperleg
https://nplus1.ru/news/2023/05/26/fingerprints-brutforce
Something went wrong
https://nplus1.ru/news/2023/05/26/menses-cardiovascular
https://nplus1.ru/news/2023/05/26/fda-approves-neuralink
https://nplus1.ru/news/2023/05/26/giardia-duodenalis
https://nplus1.ru/news/2023/05/26/vss-unity-25
https://nplus1.ru/news/2023/05/26/open-rings
https://nplus1.ru/news/2023/05/26/macrocheilia-and-crohns-disease
https://nplus1.ru/news/2023/05/26/fi

Посмотрим на несколько элементов `info`:

In [21]:
info[10:12]

Финальный штрих – импортируем `pandas` и преобразуемый полученный список кортежей в датафрейм:

In [22]:
import pandas as pd

In [23]:
df = pd.DataFrame(info)
df.head()

,0,1,2,3,4,5
0,Слоны обрадовались посетителям зоопарков,Екатерина Рощина,2023-05-26,1.5,"Зоология, Психология",А вот амфибии не в восторге Чаще всего животны...
1,В древнеримских некрополях Бейрута нашли остан...,Михаил Подрезов,2023-05-29,3.1,"Антропология, Археология",Как минимум два из 19 изученных людей происход...
2,В древнеримском унгвентарии обнаружили остатки...,Михаил Подрезов,2023-05-29,2.9,"Археология, Химия",Небольшой сосуд с парфюмом лежал в погребально...
3,Причиной крушения японского лунного модуля Hak...,Александр Войтюк,2023-05-27,1.9,Космонавтика,Зонд LRO нашел место его падения на Луну Частн...
4,Сокращение выбросов дихлорметана поможет сохра...,Михаил Бойм,2023-05-27,2.4,"Экология и климат, Химия",Дихлорметан используют для изготовления кофе б...


Добавим содержательные названия столбцов и выгрузим датафрейм в файл:

In [25]:
df.columns = ["title", "author", "date", "diffc", "rubrics", "text"]
df.head()

,title,author,date,diffc,rubrics,text
0,Слоны обрадовались посетителям зоопарков,Екатерина Рощина,2023-05-26,1.5,"Зоология, Психология",А вот амфибии не в восторге Чаще всего животны...
1,В древнеримских некрополях Бейрута нашли остан...,Михаил Подрезов,2023-05-29,3.1,"Антропология, Археология",Как минимум два из 19 изученных людей происход...
2,В древнеримском унгвентарии обнаружили остатки...,Михаил Подрезов,2023-05-29,2.9,"Археология, Химия",Небольшой сосуд с парфюмом лежал в погребально...
3,Причиной крушения японского лунного модуля Hak...,Александр Войтюк,2023-05-27,1.9,Космонавтика,Зонд LRO нашел место его падения на Луну Частн...
4,Сокращение выбросов дихлорметана поможет сохра...,Михаил Бойм,2023-05-27,2.4,"Экология и климат, Химия",Дихлорметан используют для изготовления кофе б...


**Дополнение.** Исправим тип столбца со сложностью новости – сделаем его типом `float`, чтобы с элементами можно было работать как с числами.

In [26]:
df["diffc"] = df["diffc"].astype(float)

Раз сделали столбец числовым, значит, его можно использовать для сортировки! Отсортируем строки в датафрейме от самой сложной новости к самой простой:

In [27]:
df.sort_values("diffc", ascending = False)

,title,author,date,diffc,rubrics,text
61,Две группы физиков смоделировали неабелевы эни...,Марат Хамадеев,2023-05-16,9.3,Физика,"Один компьютер — на сверхпроводящих контурах, ..."
57,Неравенства Белла без лазеек проверили на свер...,Марат Хамадеев,2023-05-11,7.8,Физика,Для этого их разнесли более чем на 30 метров Ф...
58,В распадах тау-лептонов не нашли темных бозонов,Марат Хамадеев,2023-05-13,7.6,Физика,Результат получила коллаборация Belle II Выход...
64,Аккреционный диск черной дыры воссоздали без с...,Марат Хамадеев,2023-05-22,6.9,"Физика, Астрономия",Для этого физики косо сталкивали восемь плазме...
59,«Ковер» Тальбота помог упорядоченно пленить де...,Марат Хамадеев,2023-05-13,6.8,Физика,В будущем это позволит проводить масштабные кв...
...,...,...,...,...,...,...
44,На дне Адриатического моря нашли 6900-летнюю д...,Михаил Подрезов,2023-05-09,1.3,Археология,В эпоху неолита она соединяла поселение и остр...
6,В Аттике впервые раскопали остатки дома X-IX в...,Михаил Подрезов,2023-05-27,1.2,Археология,Его нашли примерно в 60 километрах от Афин Арх...
50,В Швеции обнаружили двухметровое наскальное из...,Михаил Подрезов,2023-05-22,1.1,Археология,В общей сложности археологи открыли около 40 д...
48,В древнеримском форте нашли рельеф с изображен...,Михаил Подрезов,2023-05-20,1.1,Археология,Третья фигура в этой композиции — рог изобилия...


Топ-3 самых сложных новостей:

In [29]:
df.sort_values("diffc", ascending = False).head(3)

,title,author,date,diffc,rubrics,text
61,Две группы физиков смоделировали неабелевы эни...,Марат Хамадеев,2023-05-16,9.3,Физика,"Один компьютер — на сверхпроводящих контурах, ..."
57,Неравенства Белла без лазеек проверили на свер...,Марат Хамадеев,2023-05-11,7.8,Физика,Для этого их разнесли более чем на 30 метров Ф...
58,В распадах тау-лептонов не нашли темных бозонов,Марат Хамадеев,2023-05-13,7.6,Физика,Результат получила коллаборация Belle II Выход...


Топ-3 самых простых новостей:

In [30]:
df.sort_values("diffc").head(3)

,title,author,date,diffc,rubrics,text
50,В Швеции обнаружили двухметровое наскальное из...,Михаил Подрезов,2023-05-22,1.1,Археология,В общей сложности археологи открыли около 40 д...
48,В древнеримском форте нашли рельеф с изображен...,Михаил Подрезов,2023-05-20,1.1,Археология,Третья фигура в этой композиции — рог изобилия...
37,Восьмилетняя норвежка обнаружила кремневый кин...,Михаил Подрезов,2023-05-09,1.1,Археология,Возраст находки больше 3700 лет Восьмилетняя н...


Наконец, выгружаем датафрейм в файл:

In [31]:
df.to_excel("nplus1.xlsx")